In [ ]:
#Copyright 2019 The TensorFlow Authors.
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
import numpy as np
import matplotlib as mpl

import IPython.display as display
import PIL.Image

import tensorflow as tf
from tensorflow.keras.preprocessing import image

In [19]:
def deprocess(img):
    img = 255*(img + 1.0)/2.0
    return tf.cast(img, tf.uint8)

def show(img):
    display.display(PIL.Image.fromarray(np.array(img)))

def save(img,name):
    PIL.Image.fromarray(np.array(img)).save(name+".png","PNG")

original_img = tf.cast(np.random.randint(50, size=(512, 512, 3)), tf.uint8)
#original_img = np.array(PIL.Image.open('noise04.jpg'))


In [20]:
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

In [21]:
names = ['mixed3', 'mixed5','mixed7','mixed9']
layers = [base_model.get_layer(name).output for name in names]

dream_model = tf.keras.Model(inputs=base_model.input, outputs=layers)

In [22]:
def calc_loss(img, model):
    img_batch = tf.expand_dims(img, axis=0)
    layer_activations = model(img_batch)
    if len(layer_activations) == 1:
        layer_activations = [layer_activations]

    losses = []
    for act in layer_activations:
        loss = tf.math.reduce_mean(act)
        losses.append(loss)

    return  tf.reduce_sum(losses)#-tf.image.total_variation(img)/100000

In [28]:
def random_roll(img, maxroll):
    shift = tf.random.uniform(shape=[2], minval=-maxroll, maxval=maxroll, dtype=tf.int32)
    shift_down, shift_right = shift[0],shift[1] 
    img_rolled = tf.roll(tf.roll(img, shift_right, axis=1), shift_down, axis=0)
    return shift_down, shift_right, img_rolled

In [29]:
class TiledGradients(tf.Module):
    def __init__(self, model):
        self.model = model

    @tf.function(input_signature=(tf.TensorSpec(shape=[None,None,3], dtype=tf.float32),
                                  tf.TensorSpec(shape=[], dtype=tf.int32),  ))

    def __call__(self, img, tile_size=512,  ):      
        shift_down, shift_right, img_rolled = random_roll(img, tile_size)
        gradients = tf.zeros_like(img_rolled)
        
        xs = tf.range(0, img_rolled.shape[0], tile_size)[:-1]
        if not tf.cast(len(xs), bool):
            xs = tf.constant([0])
        ys = tf.range(0, img_rolled.shape[1], tile_size)[:-1]
        if not tf.cast(len(ys), bool):
            ys = tf.constant([0])

        for x in xs:
            for y in ys:
                with tf.GradientTape() as tape:    
                    tape.watch(img_rolled) 
                    img_tile = img_rolled[x:x+tile_size, y:y+tile_size]
                    loss = calc_loss(img_tile, self.model)
                gradients = gradients + tape.gradient(loss, img_rolled)
            gradients = tf.roll(tf.roll(gradients, -shift_right, axis=1), -shift_down, axis=0)
            gradients /= tf.math.reduce_std(gradients) + 1e-5 
        return gradients 

In [30]:
get_gradients = TiledGradients(dream_model)

In [31]:
def run_deep_dream_with_octaves(img, steps_per_octave=50, step_size=0.01, 
                                octaves=range(-3,3), octave_scale=1.5):
    base_shape = tf.shape(img)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.applications.inception_v3.preprocess_input(img)

    initial_shape = img.shape[:-1]
    img = tf.image.resize(img, initial_shape)
    resized_img = img
    for octave in octaves:
        new_size = tf.cast(tf.convert_to_tensor(base_shape[:-1]), tf.float32)*(octave_scale**octave)
        for step in range(steps_per_octave):
            resized_img = tf.image.resize(img, tf.cast(new_size, tf.int32))
            gradients = get_gradients(resized_img)
            resized_img = resized_img+gradients*step_size
            resized_img = tf.clip_by_value(resized_img, -1, 1)
                
            gradients =  tf.image.resize(gradients, initial_shape)
            img = img + gradients*step_size
            img = tf.clip_by_value(img, -1, 1)
  
    result = deprocess(img)
    return result

def run_deep_dream_with_octaves_resize(img, steps_per_octave=50, step_size=0.01, 
                                octaves=range(-3,3), octave_scale=1.5):
    base_shape = tf.shape(img)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.applications.inception_v3.preprocess_input(img)

    initial_shape = img.shape[:-1]
    for octave in octaves:
        new_size = tf.cast(tf.convert_to_tensor(base_shape[:-1]), tf.float32)*(octave_scale**octave)
        for step in range(steps_per_octave):
            img = tf.image.resize(img, tf.cast(new_size, tf.int32))
           
            gradients = get_gradients(img)
            img = img+gradients*step_size
            img = tf.clip_by_value(img, -1, 1)
    
    result = deprocess(img)
    return result

In [32]:
img = run_deep_dream_with_octaves_resize(img=original_img, steps_per_octave=50, step_size=0.01,octaves=range(-4,3), octave_scale=1.5)

